In [1]:
import os
import threading
from django.db import connection
import timeit

In [2]:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
from sync.models import StockOrder

In [4]:
StockOrder.objects.filter(id=12).update(order_quantity=0)
result=StockOrder.objects.get(id=12)

In [5]:
print("1) test id : ",result.id," order_quantity : ",result.order_quantity)

1) test id :  12  order_quantity :  0


In [19]:
def add_stock_order(product_id):
    so = StockOrder.objects.get(id=product_id)
    for _ in range(30):
        if so.order_quantity >= 0:
            so.order_quantity += 1
            so.save()
            #print("th id : ", threading.current_thread().ident, " : ", so.order_quantity,"\n")

In [20]:
thread1 = threading.Thread(target=add_stock_order, args=(result.id,))
thread2 = threading.Thread(target=add_stock_order, args=(result.id,))
thread1.start()
thread2.start()

thread1.join()
thread2.join()
result.refresh_from_db()
print(result.order_quantity)

1800034


In [21]:
# for _ in range(100):
#     add_stock_order(result.id)

# result.refresh_from_db()
# print(result.order_quantity) 

In [22]:
print("2) test id : ",result.id," order_quantity : ",result.order_quantity)

2) test id :  12  order_quantity :  1800034


In [10]:
StockOrder.objects.filter(id=12).update(order_quantity=0)

1

In [11]:
def call_update_stock(id, number):
    for _ in range(30000):
        with connection.cursor() as cursor:
            sql = f"SELECT update_quantity({id}, {number})"
            cursor.execute(sql)
            result = cursor.fetchone()  # 단일 행 결과를 가져옴
            # result = cursor.fetchall()  # 모든 행 결과를 가져옴
    
            # 쿼리 결과를 확인하고 처리할 내용을 추가로 작성하세요.
            if result is not None:
                # 쿼리 결과가 존재하는 경우에 대한 처리
                # result 변수에 반환된 결과가 저장됩니다.
                # 원하는 작업을 수행하세요.
                #print(result)
                pass
            else:
                # 쿼리 결과가 없는 경우에 대한 처리
                # 예외 상황 등에 대한 처리 코드를 작성하세요.
                print("쿼리 결과가 없습니다.")

In [12]:
call_update_stock(12,1)

In [13]:
print("3) test id : ",result.id," order_quantity : ",result.order_quantity)

3) test id :  12  order_quantity :  10


In [14]:
StockOrder.objects.filter(id=12).update(order_quantity=0)

1

In [15]:
result=StockOrder.objects.get(id=12)
print("4) test id : ",result.id," order_quantity : ",result.order_quantity)

4) test id :  12  order_quantity :  0


In [16]:
def multi_thread_t1():
    threads = []
    for i in range(10):
        thread = threading.Thread(target=call_update_stock, args=(result.id,1,))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()
    
    result.refresh_from_db()
    print(result.order_quantity)

execution_time = timeit.timeit(multi_thread_t1,number=1)

print(f'함수 실행 시간: {execution_time}초')

300000
함수 실행 시간: 48.78140710000298초


In [17]:
result=StockOrder.objects.get(id=12)
print("5) test id : ",result.id," order_quantity : ",result.order_quantity)
StockOrder.objects.filter(id=12).update(order_quantity=0)

5) test id :  12  order_quantity :  300000


1

In [18]:
def multi_thread_t2():
    threads = []
    for i in range(60):
        thread = threading.Thread(target=call_update_stock, args=(result.id,1,))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()
    
    result.refresh_from_db()
    print(result.order_quantity)

execution_time = timeit.timeit(multi_thread_t2,number=1)

print(f'함수 실행 시간: {execution_time}초')

1800000
함수 실행 시간: 358.2501574999915초
